In [1]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
import difflib 


In [2]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [3]:
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)

    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links

In [4]:
scrape_google("comment eviter le sida ?")

['https://www.tf1info.fr/sante/journee-mondiale-sexualite-sida-quels-sont-les-bons-reflexes-pour-eviter-le-vih-apres-un-rapport-non-protege-1507325.html',
 'https://www.unaids.org/fr/topic/prevention',
 'https://www.catie.ca/fr/safer-sex-guide/prevenir-linfection-par-le-vih',
 'https://www.catie.ca/fr/client-publication/sept-facons-de-prevenir-le-vih#moyens',
 'https://pvsq.org/prevenir-le-vih/',
 'https://www.catie.ca/fr/client-publication/sept-facons-de-prevenir-le-vih',
 'https://lasante.net/fiches-conseil/infos-pratiques/au-quotidien/prevention-sida-vih.htm',
 'https://www.catie.ca/fr/client-publication/sept-facons-de-prevenir-le-vih#prep',
 'https://cliniquelactuel.com/Comment_eviter_d_attraper_le_VIH_sida',
 'http://www.actionpositive.ca/guide-vih/prevention/',
 'https://www.ameli.fr/assure/sante/themes/vih/prevention',
 'https://www.unaids.org/fr/frequently-asked-questions-about-hiv-and-aids']

In [5]:
def get_results(query):
    
    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.fr/search?q=" + query)
    
    return response

In [6]:
def parse_results(response):
    
    css_identifier_result = ".tF2Cxc"
    css_identifier_title = "h3"
    css_identifier_link = ".yuRUbf a"
    css_identifier_text = ".VwiC3b"
    #title = response.html.find('title')
    #print(title)
    #print(title[0].text)
    results = response.html.find(css_identifier_result)
    output = []
    
    for result in results:###

        item = {
            'title': result.find(css_identifier_title, first=True).text,
            'link': result.find(css_identifier_link, first=True).attrs['href'],
            'text': result.find(css_identifier_text, first=True).text
        }
        
        output.append(item)
        
    return output

In [7]:
def parse_results1(response):
    
    css_identifier_result = ".tF2Cxc"
    css_identifier_title = "h3"
    css_identifier_link = ".yuRUbf a"
    css_identifier_text = ".VwiC3b"
    #title = response.html.find('title')
    #print(title)
    #print(title[0].text)
    results = response.html.find(css_identifier_result)
    output = []
    for result in results:###
        if(result.find(css_identifier_text, first=True)!=None):
            item = {
                'title': result.find(css_identifier_title, first=True).text,
                'link': result.find(css_identifier_link, first=True).attrs['href'],
                'text': result.find(css_identifier_text, first=True).text
            }
        
            output.append(item)
        
    return output

In [8]:
def google_search(query):
    response = get_results(query)
    return parse_results(response)

In [9]:
def google_search1(query):
    response = get_results(query)
    return parse_results1(response)

In [10]:
#results = google_search("peut on ressentir du plaisir avec un petit penis ?")
#results

In [11]:
def find_faq(question):
    results = google_search1(question)
    output=str(results[0]["text"]).replace("\xa0"," ")
    output=output.replace("janv.", " ")
    output=output.replace('févr.', ' ')
    output=output.replace('avr.', ' ')
    output=output.replace('juill.', ' ')
    output=output.replace("sept.", " ")
    output=output.replace('oct.', ' ')
    output=output.replace('nov.', ' ')
    output=output.replace('déc.', ' ')
    text=""
    #print(output)
    i=0
    tr=0
    for e in range(0, len(output)):
        if output[e].isupper():
            if tr==1:
                text=""
            i=1
        if output[e]==".":
            if str("?") in str(text): 
                text=""
                tr=1
            else:
                if(len(text)>4):
                    i=0
                    text=text+"..."
                    break
                else:
                    text=""
                    tr=1
        if(i==1):
            text=text+output[e]
    return text, results[0]["link"]

In [12]:
def test_faq(question):
    question=question.lower()
    splits= question.split(" ")
    word=['comment','c\'est quoi','quel','quoi','qu\'est ce que','quels','quelle','quelles','pourquoi']
    keyword=['ist','mst','sodomie','vih','sexe','doigter','clitoris','penis','syphilis','chatte','minou','vagin','baiser','anus',
             'poils','couille','bouton','mycose','hepatite','pisse chaude','prevention','capote','condom','eviter','preservatif',
            'enceinte','sperme','bebe','ecographie','cancer','testicules','seins','nichons','branlette espagnol','levrette',
            'missionnaire','position','kamasutra','orgasme','jouir','sida','enculer','sucer','cunnilingus','masturbation',
            'caresser','eviter','bébé','enfant','accoucher','faire l\'amour','rapport sexuel','pilule','hygiene',
            'epilation','raser','saigner','regle','lendemain','regle','planification familial','puceau','premiere fois','douleur'
            ,'ventre','buccale','ovaire','spermatozoide','preliminaire','liquide', 'cyprine','mouillé','prepuce','falange'
            ,'gode','vibro','lecher','plaisir','protection','éjaculation','précoce','retenir','sauce','abstinence','serment',
            'saignement','sang','nettoyer','laver','anal','mal','proteger','infection','maladie','bacterie','propager','etroit',
            'devierger','depuceler','traitement','médicament','sterilet','pharmacie','gynecologue','consultation','contamination',
            'transmission','sterile','inpuissance','symptome','infectueux','ovaire','trompe','plan cul','coucher','avorter',
            'complication','uterus','vulve','urine','vessie','depistage','partenaire','danger','circonsision','circonsis','point g',
            'branlette','oral','vaccin','immunité','contaminer','embrasser','salive','gorge profonde','lesbienne','gay',
            'fisting','sadomaso','sm','fétichisme','masochiste','trans','transexuelle','ladyboy','grossesse','avortement',
            'déchirer','hymen','branler','crier','signe','syndrome','niquer']
    i=0
    for q in splits:
        x=difflib.get_close_matches(q, word)
        if(len(x)>0):
            i=1
    if(i==0):
        return True
    else :
        i=0
        for q in splits: 
            x=difflib.get_close_matches(q, keyword)
            if(len(x)>0):
                i=1
        if(i==0):
            return True
        else:
            return False

In [13]:
def faq(question):
    if(test_faq(question)==False):
        return find_faq(question)
    else:
        return None, None

In [14]:
x, y = faq("c'est quoi la branlette?")
if (x==None):
    print("Votre question est invalide")
    print("Veuillez reformuler svp")
else:
    print("reponse : ", x)
    print("lien : ", y )

reponse :  Branlette est un terme très familier, voire vulgaire, que l'on utilise pour désigner la masturbation réalisée par l'homme...
lien :  https://www.linternaute.fr/dictionnaire/fr/definition/branlette/


In [15]:
from flask import Flask, request, jsonify
from flask_restful import Resource, Api, reqparse
import pandas as pd
from flask_cors import CORS
import json
app = Flask(__name__)
cors = CORS(app, resources={r"*": {"origins": "*"}})
api= Api(app)

In [16]:
# POur recuperer la reponse de la faq
class Faq(Resource):
    def get(self, question):
   
         #recuperation de la faq
        temp1="None"
        temp2="None"
        
        temp1, temp2 = faq(question)
        
        if(temp1==None and temp2==None):
            q=question.lower()
            q=q.replace(" ","")
          
            if(q=="lamascotteestjolie"):
                temp1="merci"
                temp2=" "
                print(q)
        base = {'reponse': temp1,'lien':temp2}
        
            
        return  jsonify(base)


In [17]:
##
## Actually setup the Api resource routing here
##
api.add_resource(Faq, '/faq/<question>')


In [ ]:
if __name__ == '__main__':
    app.run()  # run our Flask app

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Dec/2022 06:49:00] "GET /faq/c'est%20quoi%20les%20risques%20du%20sida%20 HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 06:49:19] "GET /faq/c'est%20quoi%20un%20mst%20 HTTP/1.1" 200 -
